# Prepare script for free_energy calculations

This Notebook is designed to prepare free_energy calculations basd on finished relax_dynamic:at_temp calculations. 

The actions in this Notebook may get replaced with a built-in buildcombos function later on...


In [1]:
import iprPy
import numpy as np
import atomman as am

In [2]:
database_name = 'master'
run_directory_name = 'master_5'

lammps_command = 'lmp_mpi'

family = 'A1--Cu--fcc'
sizemults = '10 10 10'

rtol = 0.05
crystal_family = 'cubic'

In [3]:
master = iprPy.load_database(database_name)
calc = iprPy.load_calculation('free_energy')

In [4]:
def check_crystal_family(df, crystal_family, atol=0.0, rtol=0.05):
    """
    Checks if the 'box' field of df, which should be an atomman.Box object,
    is of the indicated crystal family within rtol.
    """
    
    # Define box_parameters apply function based on crystal_family
    
    if crystal_family == 'cubic':
        def box_parameters(series):
            """Check values and build the box_parameter term for cubic systems"""
            if series.box.iscubic(atol=atol, rtol=rtol):
                a = np.mean([series.box.a, series.box.b, series.box.c])
                return f'{a} {a} {a}'
            else:
                return np.nan
    
    elif crystal_family == 'hexagonal':
        def box_parameters(series):
            """Check values and build the box_parameter term for hexagonal systems"""
            if series.box.ishexagonal(atol=atol, rtol=rtol):
                a = np.mean([series.box.a, series.box.b])
                c = series.box.c
                return f'{a} {a} {c} 90.0 90.0 120.0'
            else:
                return np.nan
            
    elif crystal_family == 'tetragonal':     
        def box_parameters(series):
            """Check values and build the box_parameter term for tetragonal systems"""
            if series.box.istetragonal(atol=atol, rtol=rtol):
                a = np.mean([series.box.a, series.box.b])
                c = series.box.c
                return f'{a} {a} {c}'
            else:
                return np.nan
    
    elif crystal_family == 'rhombohedral':
        def box_parameters(series):
            """Check values and build the box_parameter term for rhombohedral systems"""
            if series.box.isrhombohedral(atol=atol, rtol=rtol):
                a = np.mean([series.box.a, series.box.b, series.box.c])
                alpha = np.mean([series.box.alpha, series.box.beta, series.box.gamma])
                return f'{a} {a} {a} {alpha} {alpha} {alpha}'
            else:
                return np.nan
        
    elif crystal_family == 'orthorhombic':
        def box_parameters(series):
            """Check values and build the box_parameter term for orthorhombic systems"""
            if series.box.isorthorhombic(atol=atol, rtol=rtol):
                a = series.box.a
                b = series.box.b
                c = series.box.c
                return f'{a} {b} {c}'
            else:
                return np.nan
        
    elif crystal_family == 'monoclinic':
        def box_parameters(series):
            """Check values and build the box_parameter term for monoclinic systems"""
            if series.box.ismonoclinic(atol=atol, rtol=rtol):
                a = series.box.a
                b = series.box.b
                c = series.box.c
                beta = series.box.beta
                return f'{a} {b} {c} 90.0 {beta} 90.0'
            else:
                return np.nan
            
    elif crystal_family == 'triclinic':
        def box_parameters(series):
            """Check values and build the box_parameter term for triclinic systems"""
            if series.box.istriclinic(atol=atol, rtol=rtol):
                a = series.box.a
                b = series.box.b
                c = series.box.c
                alpha = series.box.alpha
                beta = series.box.beta
                gamma = series.box.gamma
                return f'{a} {b} {c} {alpha} {beta} {gamma}'
            else:
                return np.nan
            
    else:
        raise ValueError(f'unknown crystal family: {crystal_family}')
    
    # Use the apply function on the dataframe
    df['box_parameters'] = df.apply(box_parameters, axis=1)
        
    # Return only the dataframe entries with good box_parameters
    return df[df.box_parameters.notna()]

In [5]:
# Fetch finished relax_dynamic records
records_df = master.get_records_df('calculation_relax_dynamic', branch='at_temp', status='finished',
                                   family=family,
                                   #potential_LAMMPS_id = '1999--Mishin-Y--Al--LAMMPS--ipr1',
                                  ).sort_values(['potential_LAMMPS_id', 'temperature'])

# Build box and filter based on crystal family check
def make_box(series):
    """Create cell Box from relaxed lengths and tilts"""
    return am.Box(lx=series.lx, ly=series.ly, lz=series.lz,
                  xy=series.xy, xz=series.xz, yz=series.yz)
records_df['box'] = records_df.apply(make_box, axis=1)
good_df = check_crystal_family(records_df, crystal_family, rtol=rtol)

good_df[['potential_LAMMPS_id', 'temperature', 'symbols', 'box_parameters']]

,potential_LAMMPS_id,temperature,symbols,box_parameters
224,1986--Foiles-S-M--Ag--LAMMPS--ipr1,100.0,Ag,4.097820104177621 4.097820104177621 4.09782010...
361,1986--Foiles-S-M--Ag--LAMMPS--ipr1,150.0,Ag,4.101760244932128 4.101760244932128 4.10176024...
22,1986--Foiles-S-M--Ag--LAMMPS--ipr1,200.0,Ag,4.105824782094095 4.105824782094095 4.10582478...
34,1986--Foiles-S-M--Ag--LAMMPS--ipr1,250.0,Ag,4.109967286230813 4.109967286230813 4.10996728...
51,1986--Foiles-S-M--Ag--LAMMPS--ipr1,300.0,Ag,4.114189975946421 4.114189975946421 4.11418997...
...,...,...,...,...
263,2022--Mahata-A--Al-Ni--LAMMPS--ipr1,1850.0,Ni,3.6073160578151984 3.6073160578151984 3.607316...
237,2022--Mahata-A--Al-Ni--LAMMPS--ipr1,1900.0,Ni,3.61116616315509 3.61116616315509 3.6111661631...
49,2022--Mahata-A--Al-Ni--LAMMPS--ipr1,1950.0,Ni,3.615268280693984 3.615268280693984 3.61526828...
45,2022--Mahata-A--Al-Ni--LAMMPS--ipr1,2000.0,Ni,3.619953841765829 3.619953841765829 3.61995384...


In [6]:
# Construct prepare keys for each finished calculation
prepare_keys = {}
prepare_keys['lammps_command'] = lammps_command
prepare_keys['sizemults'] = sizemults
prepare_keys['equilsteps'] = '25000'
prepare_keys['switchsteps'] = '50000'
prepare_keys['springsteps'] = '50000'

prepare_keys['potential_file'] = []
prepare_keys['potential_content'] = []
prepare_keys['potential_dir'] = []
prepare_keys['potential_dir_content'] = []
prepare_keys['load_file'] = []
prepare_keys['load_content'] = []
prepare_keys['load_style'] = []
prepare_keys['symbols'] = []
prepare_keys['box_parameters'] = []
prepare_keys['temperature'] = []

for i in good_df.index:
    series = good_df.loc[i]
    prepare_keys['potential_file'].append(f'{series.potential_LAMMPS_id}.json')
    prepare_keys['potential_content'].append(f'record {series.potential_LAMMPS_id}')
    prepare_keys['potential_dir'].append(series.potential_LAMMPS_id)
    prepare_keys['potential_dir_content'].append(f'tar {series.potential_LAMMPS_id}')
    prepare_keys['load_file'].append(f'{series.family}.json')
    prepare_keys['load_content'].append(f'record {series.family}')
    prepare_keys['load_style'].append('system_model')
    prepare_keys['symbols'].append(series.symbols)
    prepare_keys['box_parameters'].append(series.box_parameters)
    prepare_keys['temperature'].append(str(series.temperature))

In [7]:
# Prepare
master.prepare(run_directory_name, 'free_energy', **prepare_keys)

130 existing calculation records found
278 calculation combinations to check
139 new records to prepare


preparing: 100%|#####################################################################| 139/139 [06:38<00:00,  2.87s/it]
